In [1]:
from layers import SelfAttention
import pandas as pd
import argparse
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense

import sys
# add parent directory to Python path for layers.py access
sys.path.append('..')
splits = {'train': 'data/train-00000-of-00001.parquet',
          'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
train_df = pd.read_parquet(
    "hf://datasets/google-research-datasets/poem_sentiment/" + splits["train"])
validation_df = pd.read_parquet(
    "hf://datasets/google-research-datasets/poem_sentiment/" + splits["validation"])
test_df = pd.read_parquet(
    "hf://datasets/google-research-datasets/poem_sentiment/" + splits["test"])
# verse_text, label
# label: 0 = negative, 1 = positive, 2 = no_impact, 3 = mixed

c:\Users\alexi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Argument specification
parser = argparse.ArgumentParser()
parser.add_argument("--config",
                    default=0,
                    help="Integer value representing a model configuration")
# CONFIG OPTIONS:
# 0: Simple Multi-Layer Perceptron Model
# 1: Simple Multi-Layer Perceptron Model w/ Self-Attention (Non-Penalized)
# 2: Simple Multi-Layer Perceptron Model w/ Self-Attention (Penalized)


_StoreAction(option_strings=['--config'], dest='config', nargs=None, const=None, default=0, type=None, choices=None, required=False, help='Integer value representing a model configuration', metavar=None)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

X_train, Y_train, X_validation, Y_validation = train_df['verse_text'].values, train_df['label'].values, validation_df['verse_text'].values, validation_df['label'].values

sequence_length = 900
vocabulary_size = 10000  # choose 10k most-used words for truncated vocabulary

# Tokenize the text
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_validation = tokenizer.texts_to_sequences(X_validation)

X_train = pad_sequences(sequences=X_train, maxlen=sequence_length)
X_validation = pad_sequences(sequences=X_validation, maxlen=sequence_length)

In [12]:
def build_Attention_Model(config,vocabulary_size, sequence_length, embedding_dims, batch_size, num_epochs, random_seed, X_train, Y_train, X_validation, Y_validation):
    tf.random.set_seed(random_seed)
    X = Input(shape=(sequence_length,), batch_size=batch_size)

    embedded = Embedding(input_dim=vocabulary_size, output_dim=embedding_dims)(X)

    # Optional Self-Attention Mechanisms
    if config == 1:
        embedded, attention_weights = SelfAttention(size=50,
                                                    num_hops=6,
                                                    use_penalization=False)(embedded)
    elif config == 2:
        embedded, attention_weights = SelfAttention(size=50,
                                                    num_hops=6,
                                                    use_penalization=True,
                                                    penalty_coefficient=0.1)(embedded)

    # Multi-Layer Perceptron
    embedded_flattened = Flatten()(embedded)
    fully_connected = Dense(units=250, activation='relu')(embedded_flattened)

    # Prediction Layer
    Y = Dense(units=1, activation='sigmoid')(fully_connected)

    # Compile model
    model = Model(inputs=X, outputs=Y)
    model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

#First Model with 0 config

In [15]:
random_seed = 42
# choose 1000-word sequences, either pad or truncate sequences to this
embedding_dims = 50      # number of dimensions to represent each word in vector space
batch_size = 100         # feed in the neural network in 100-example training batches
num_epochs = 100
config = 0

In [ ]:
perceptron = build_Attention_Model(config,vocabulary_size, sequence_length, embedding_dims, batch_size, num_epochs, random_seed, train_df['verse_text'], train_df['label'], validation_df['verse_text'], validation_df['label'])

In [ ]:
perceptron.fit(x=X_train, y=Y_train,
          validation_data=(X_validation, Y_validation),
          epochs=num_epochs, batch_size=batch_size)